In [1]:
import csv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from os import path

import random
import fnmatch
import datetime
import pickle

# data processing
import numpy as np
np.set_printoptions(formatter={'float_kind':lambda x: "%.4f" % x})

import pandas as pd
pd.set_option('display.width', 300)
pd.set_option('display.float_format', '{:,.4f}'.format)
pd.set_option('display.max_colwidth', 200)

# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
print( f'tf.__version__: {tf.__version__}' )
print( f'keras.__version__: {keras.__version__}' )

# sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# imaging
import cv2
from imgaug import augmenters as img_aug
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from PIL import Image, ImageEnhance


tf.__version__: 2.4.1
keras.__version__: 2.4.0


In [2]:
y = []
x_names = []

with open('training_norm.csv', mode='r') as infile:
    reader = csv.reader(infile)
    next(reader)
#     print(reader)
#         c = 0
     for rows in reader:
#            if c > 100:
#                break
        f = rows[0]
        a = float(rows[1])
        s = float(rows[2])
        y.append([a,s])
        filename = str(f) + '.png'
        x_names.append(filename)
            

In [3]:
def image_data_generator(image_paths, steering_angles, batch_size, is_training):
    while True:
        batch_images = []
        batch_steering_angles = []
        path = '/Users/Kamyab/Documents/UoN/MLiS_2/Project/Data/objects'
       
        for i in range(batch_size):
            random_index = random.randint(0, len(image_paths) - 1)
           
            image_path = path + image_paths[random_index]
            image = cv2.imread(image_path)
            steering_angle = steering_angles[random_index]
            if is_training:
                # training: augment image
                image, steering_angle = random_augment(image, steering_angle)
             
            image = img_preprocess(image)
            batch_images.append(image)
            batch_steering_angles.append(steering_angle)
           
        yield( np.asarray(batch_images), np.asarray(batch_steering_angles))

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split( x_names, y, test_size=0.2)
print("Training data: %d\nValidation data: %d" % (len(X_train), len(X_valid)))

Training data: 11034
Validation data: 2759


In [5]:
def zoom(image):
    zoom = img_aug.Affine(scale=(1, 1.3))  # zoom from 100% (no zoom) to 130%
    image = zoom.augment_image(image)
    return image

In [6]:
def pan(image):
    # pan left / right / up / down about 10%
    pan = img_aug.Affine(translate_percent= {"x" : (-0.1, 0.1), "y": (-0.1, 0.1)})
    image = pan.augment_image(image)
    return image

In [7]:
def adjust_brightness(image):
    # increase or decrease brightness by 30%
    brightness = img_aug.Multiply((0.7, 1.3))
    image = brightness.augment_image(image)
    return image

In [8]:
def blur(image):
    kernel_size = random.randint(1, 5)  # kernel larger than 5 would make the image way too blurry
    image = cv2.blur(image,(kernel_size, kernel_size))
   
    return image

In [9]:
def random_flip(image, steering_angle):
    is_flip = random.randint(0, 1)
    if is_flip == 1:
        # randomly flip horizon
        image = cv2.flip(image,1)
        steering_angle[0] = (80 - steering_angle[0]*80)/80
        #(180 - (steering_angle[0]*80 + 50) -50)/80
   
    return image, steering_angle


In [10]:
def random_augment(image, steering_angle):
    if np.random.rand() < 0.5:
        image = pan(image)
    if np.random.rand() < 0.5:
        image = zoom(image)
    if np.random.rand() < 0.5:
        image = blur(image)
    if np.random.rand() < 0.5:
        image = adjust_brightness(image)
    image, steering_angle = random_flip(image, steering_angle)
    
    return image, steering_angle



# show a few randomly augmented images
# ncol = 2
# nrow = 10
# fig, axes = plt.subplots(nrow, ncol, figsize=(15, 50))

# for i in range(nrow):
#     rand_index = random.randint(0, len(image_paths) - 1)
#     image_path = image_paths[rand_index]
#     steering_angle_orig = steering_angles[rand_index]
    
#     image_orig = my_imread(image_path)
#     image_aug, steering_angle_aug = random_augment(image_orig, steering_angle_orig)
    
#     axes[i][0].imshow(image_orig)
#     axes[i][0].set_title("original, angle=%s" % steering_angle_orig)
#     axes[i][1].imshow(image_aug)
#     axes[i][1].set_title("augmented, angle=%s" % steering_angle_aug)
    

In [11]:
def img_preprocess(image):
    height, _, _ = image.shape
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (160,120)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing
    return image

In [13]:
# def nvidia_model():
#     model = Sequential(name='Nvidia_Model')
    
#     # elu=Expenential Linear Unit, similar to leaky Relu
#     # skipping 1st hiddel layer (nomralization layer), as we have normalized the data
    
#     # Convolution Layers
#     model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu')) 
#     model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu')) 
#     model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu')) 
#     model.add(Conv2D(64, (3, 3), activation='elu')) 
#     model.add(Dropout(0.2)) # not in original model. added for more robustness
#     model.add(Conv2D(64, (3, 3), activation='elu')) 
    
#     # Fully Connected Layers
#     model.add(Flatten())
#     model.add(Dropout(0.2)) # not in original model. added for more robustness
#     model.add(Dense(100, activation='elu'))
#     model.add(Dense(50, activation='elu'))
#     model.add(Dense(10, activation='elu'))
#     model.add(Dense(2, activation='tanh')) 
#     model.add(Dense(2, activation='sigmoid')) 

    
#     # output layer: turn angle (from 45-135, 90 is straight, <90 turn left, >90 turn right)
#     model.add(Dense(1))
#     model.add(Dense(2))
    
#     # since this is a regression problem not classification problem,
#     # we use MSE (Mean Squared Error) as loss function
#     optimizer = Adam(lr=1e-3) # lr is learning rate
#     model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    
#     return model

# model = nvidia_model()
# print(model.summary())

Model: "Nvidia_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 3, 20, 64)         27712     
_________________________________________________________________
dropout_4 (Dropout)          (None, 3, 20, 64)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 1152)             

In [16]:
def nvidia_model():
    model = Sequential(name='Nvidia_Model')
    
    # elu=Expenential Linear Unit, similar to leaky Relu
    # skipping 1st hiddel layer (nomralization layer), as we have normalized the data
    
    # Convolution Layers
    model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(120, 160, 3), activation='elu')) 
    model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    
    # output layer: turn angle (from 45-135, 90 is straight, <90 turn left, >90 turn right)
    model.add(Dense(2, activation='sigmoid'))#, activation='tanh')) 
    
    # since this is a regression problem not classification problem,
    # we use MSE (Mean Squared Error) as loss function
    optimizer = Adam(lr=1e-3) # lr is learning rate
    model.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
 

 

    return model

 

model = nvidia_model()
print(model.summary())

Model: "Nvidia_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 58, 78, 24)        1824      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 37, 36)        21636     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 17, 48)        43248     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 10, 15, 64)        27712     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 15, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 13, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 6656)             

In [ ]:
# saves the model weights after each epoch if the validation loss decreased
model_output_dir = '/Users/Kamyab/Documents/'

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(model_output_dir,'lane_navigation_check.h5'), verbose=1, save_best_only=True)

history = model.fit_generator(image_data_generator( X_train, y_train, batch_size=100, is_training=True),
                              steps_per_epoch=300,
                              epochs=20,
                              validation_data = image_data_generator( X_valid, y_valid, batch_size=100, is_training=False),
                              validation_steps=200,
                              verbose=1,
                              shuffle=1,
                              callbacks=[checkpoint_callback])
# always save model output as soon as model finishes training
model.save(os.path.join(model_output_dir,'lane_navigation_final.h5'))

Epoch 1/20
300/300 [==============================] - 323s 1s/step - loss: 0.1198 - accuracy: 0.7411 - val_loss: 0.0555 - val_accuracy: 0.9116

Epoch 00001: val_loss improved from inf to 0.05549, saving model to /Users/Kamyab/Documents/lane_navigation_check.h5
Epoch 2/20
300/300 [==============================] - 327s 1s/step - loss: 0.0564 - accuracy: 0.9087 - val_loss: 0.0442 - val_accuracy: 0.9362

Epoch 00002: val_loss improved from 0.05549 to 0.04417, saving model to /Users/Kamyab/Documents/lane_navigation_check.h5
Epoch 3/20
300/300 [==============================] - 338s 1s/step - loss: 0.0476 - accuracy: 0.9252 - val_loss: 0.0413 - val_accuracy: 0.9427

Epoch 00003: val_loss improved from 0.04417 to 0.04134, saving model to /Users/Kamyab/Documents/lane_navigation_check.h5
Epoch 4/20
300/300 [==============================] - 340s 1s/step - loss: 0.0433 - accuracy: 0.9325 - val_loss: 0.0333 - val_accuracy: 0.9514

Epoch 00004: val_loss improved from 0.04134 to 0.03333, saving mo

In [14]:
model = load_model(os.path.join(model_output_dir,'lane_navigation_final.h5'))
       
def compute_steering_angle(self, frame):
    preprocessed = img_preprocess(frame)
    X = np.asarray([preprocessed])
    steering_angle = model.predict(X)[0]
    return steering_angle

In [19]:
path_file = '/Users/Kamyab/Documents/UoN/MLiS_2/Project/Data/test_data/test_data/'
test_names = np.arange(1020) + 1
with open(path_file + 'submission.csv', mode='w', newline='') as submission:
    writer = csv.writer(submission, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['image_id', 'angle', 'speed'])
    for name in test_names:     
        img = cv2.imread(path_file + str(name) + '.png')
#         print(img)
        prediction = compute_steering_angle(model, img)
        if prediction[1] < 0.5:
            prediction[1] = 0
        else:
            prediction[1] = 1
        writer.writerow([name, prediction[0], prediction[1]])
        print(name, prediction[0], prediction[1])
   
 

1 0.5287911 6.8683084e-06
2 0.5059447 0.9892997
3 0.5057718 0.9848819
4 0.505948 0.9893698
5 0.506156 0.9929985
6 0.50589436 0.9881656
7 0.50815374 0.9998765
8 0.5053856 0.96753687
9 0.51807797 0.0014179647
10 0.5046087 0.8607653
11 0.5273309 1.4553915e-05
12 0.5216904 0.00022932887
13 0.5053503 0.9652157
14 0.52962995 9.11681e-06
15 0.50551605 0.9748837
16 0.5073282 0.1028983
17 0.5306032 4.7426223e-09
18 0.54589874 2.183717e-08
19 0.5205026 0.00062802434
20 0.50497615 0.9286219
21 0.5054987 0.97400856
22 0.5050016 0.9319617
23 0.504764 0.89437145
24 0.5403263 1.9741086e-07
25 0.5114664 0.017750949
26 0.5071989 0.9991474
27 0.53080416 1.7894512e-06
28 0.5343976 2.5369007e-10
29 0.53252757 3.3623578e-06
30 0.5054592 0.97190666
31 0.5041508 0.70984334
32 0.5056534 0.9808643
33 0.5301675 7.920233e-06
34 0.50605345 0.9913962
35 0.51549566 0.005169362
36 0.50563866 0.98029566
37 0.5066173 0.9972369
38 0.50611544 0.99240327
39 0.5323948 1.8333567e-06
40 0.50604504 0.99124944
41 0.50547254 0